In [ ]:
print("okg")

In [ ]:
%pwd


In [ ]:
import os
os.chdir("../")

In [ ]:
%pwd #project working dir

In [ ]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
def load_pdf_file(data):
    loader= DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents= loader.load()
    return documents

In [ ]:
extracted_data= load_pdf_file(data='Data/')

In [ ]:
#extracted_data


In [ ]:
def text_split(extracted_data):
    text_splitter= RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks= text_splitter.split_documents(extracted_data)
    return text_chunks

In [ ]:
text_chunks= text_split(extracted_data)
print("Length of text chunks",len(text_chunks))

In [ ]:
#text_chunks


In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

In [ ]:
def download_hugging_face_embeddings():
    embeddings= HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [ ]:
embeddings= download_hugging_face_embeddings()

In [ ]:
query_result= embeddings.embed_query("Hello World")
print("Length", len(query_result))

In [ ]:
#query_result


In [67]:
from dotenv import load_dotenv
load_dotenv()

True

In [68]:
PINECONE_API_KEY= os.environ.get('PINECONE_API_KEY')
TOGETHER_API_KEY= os.environ.get('TOGETHER_API_KEY')

In [ ]:
from pinecone import Pinecone, ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "mediguide-index"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [69]:
import os
os.environ["PINECONE_API_KEY"]= PINECONE_API_KEY
os.environ["TOGETHER_API_KEY"]= TOGETHER_API_KEY

In [ ]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [ ]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [ ]:
docsearch

In [90]:
retriever = docsearch.as_retriever(search_type= "similarity", search_kwargs= {"k":5})

In [91]:
retrieved_docs = retriever.invoke("What is Acne?")

In [92]:
retrieved_docs

[Document(id='725cd052-5e47-46fd-9176-9a1102d6b450', metadata={'author': 'EDGE Note', 'creationdate': '2009-08-20T02:28:03-05:00', 'creator': 'PScript5.dll Version 5.2.2', 'moddate': '2009-08-20T02:28:03-05:00', 'page': 650.0, 'page_label': '651', 'producer': 'Acrobat Distiller 9.0.0 (Windows)', 'source': 'Data/c46528ba033a8197e32c40887c398198.pdf', 'title': 'Principles of internal medicine', 'total_pages': 5113.0}, page_content="ACNE (Table 57-7)\nAcne vulgaris and acne rosacea are the two major forms of acne (Chap. 56). Estrogens \ndecrease sebaceous gland activity, whereas androgens enhance sebum production. \nTherefore, acne vulgaris in an adult, especially if it is of recent onset, may be a \nreflection of increased levels of circulating androgens. Dysfunction of the ovary or \nadrenal gland, e.g., polycystic ovary disease or Cushing's syndrome, can lead to the"),
 Document(id='28be37f2-5322-49c5-bd53-311cd7865767', metadata={'author': 'EDGE Note', 'creationdate': '2009-08-20T02:2

In [108]:
from langchain_together import Together


# Initialize the Together LLM
llm = Together(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
    temperature=0.4,
    max_tokens= 700
)

In [109]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


# Define the system prompt
system_prompt = (
    "You are an assistant for question-answering tasks for offering patients text-based responses"
"to symptom-related questions, while clarifying that it does not replace professional diagnosis."
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use four sentences maximum and keep the "
    "answer concise and professional. Do not repeat same sentences.\n\n{context}"
)

# Create the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])

In [110]:

# Create the question-answering chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)

# Assume 'retriever' is already defined
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# Invoke the chain with a sample question


In [112]:
response = rag_chain.invoke({"input": "I am bleeding from ears, what is it?"})
print(response["answer"])

 
Assistant

Bleeding from the ears can be caused by various conditions, including trauma, infection, or a perforated eardrum. It's also possible that you may have a condition such as otitis externa, also known as swimmer's ear, which can cause bleeding due to infection or inflammation. I recommend consulting a healthcare professional for a proper diagnosis and treatment. They will be able to assess your symptoms and provide a more accurate diagnosis.
